<a href="https://colab.research.google.com/github/aravali324/project/blob/master/recurrent_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


from keras.applications.resnet50 import conv_block, identity_block
from keras.layers import (Activation, BatchNormalization, Convolution2D, Dense,
                          Flatten, Input, MaxPooling2D, ZeroPadding2D)
from keras.models import Model
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical

TensorFlow Version: 1.15.0


In [0]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data = []
labels = []
for root, dirt, files in os.walk("/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training"):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            imagePath = os.path.join(root, file)
            image = cv2.imread(imagePath)
            label = os.path.basename(root).replace(" ", "_").lower()
            image = cv2.resize(image, (224, 224))
            data.append(image)
            labels.append(label)
            print(label)
            print(imagePath)


2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_01.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_011.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_010.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_015.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_018.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_013.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_016.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_012.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_014.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_017.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/Training/2/B_019.jpg
2
/content/gdrive/My Drive/Colab Notebooks/project_recu

In [0]:
# load dataset
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/project_recurrentplot/training_excel1 (1).csv")

In [0]:
df.head

<bound method NDFrame.head of       a      b     c  d  e  f  ...    xx    yy    zz    aaa   bbb  condition
0    57  167.0  53.0  0  0  0  ...   8.4  1.11  1.15  135.5  83.5          1
1    57  167.0  53.0  0  0  0  ...   8.4  1.11  1.15  135.5  83.5          1
2    57  167.0  53.0  0  0  0  ...   8.4  1.11  1.15  135.5  83.5          1
3    57  167.0  53.0  0  0  0  ...   8.4  1.11  1.15  135.5  83.5          1
4    57  167.0  53.0  0  0  0  ...   8.4  1.11  1.15  135.5  83.5          1
..   ..    ...   ... .. .. ..  ...   ...   ...   ...    ...   ...        ...
275  73  157.0  62.0  1  1  1  ...  10.4  0.74  0.87  148.0  70.0          4
276  73  157.0  62.0  1  1  1  ...  10.4  0.74  0.87  148.0  70.0          4
277  73  157.0  62.0  1  1  1  ...  10.4  0.74  0.87  148.0  70.0          4
278  73  157.0  62.0  1  1  1  ...  10.4  0.74  0.87  148.0  70.0          4
279  73  157.0  62.0  1  1  1  ...  10.4  0.74  0.87  148.0  70.0          4

[280 rows x 55 columns]>

In [0]:
target=df['condition']
df.drop('condition',axis=1,inplace=True)# Normalization
df1 = (df - df.mean()) / (df.max() - df.min()+0.0001)

In [0]:
df1.head(10)

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,kk,ll,mm,nn,oo,pp,qq,rr,ss,tt,uu,vv,ww,xx,yy,zz,aaa,bbb
0,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
1,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
2,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
3,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
4,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
5,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
6,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-0.042017,0.005102,-0.550124,0.023109,-0.209821,-0.178565,0.0,-0.321413,-0.083331,-0.130948,0.00188,-0.571371,0.002904,-0.035714,0.151614,0.051017,0.107705,0.10025
7,-0.348639,0.110317,-0.135366,-0.49995,-0.642793,-0.428529,-0.357107,-0.071421,0.892768,-0.142843,0.0,-0.071421,-0.214264,-0.122449,0.120535,0.127034,-0.285686,0.34729,-0.32653,-0.123258,-0.276726,-0.054908,-0.182895,0.162817,-0.179811,-0.218379,-0.857057,-0.137023,-0.107714,-0.032161,-0.142843,-0.142843,0.0,-0.071421,0.403725,0.613298,-

In [0]:
target.head()

0    1
1    1
2    1
3    1
4    1
Name: condition, dtype: int64

In [0]:
data = np.array(data, dtype="float") / 255.0

In [0]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("[INFO] processing data...")
split = train_test_split(df1, data,target,  test_size=0.10, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX, trainlabel, testlabel) = split

[INFO] processing data...


In [0]:
le = LabelEncoder()
labels_train = le.fit_transform(trainlabel)
labels_train = np_utils.to_categorical(labels_train, 4)

le = LabelEncoder()
labels_test = le.fit_transform(testlabel)
labels_test = np_utils.to_categorical(labels_test, 4)

In [0]:
def create_segments_and_labels(df, time_steps, step):

    # x, y, z acceleration as features
    N_FEATURES = 54
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df), step):
        x1 = df['a'].values[i: i + time_steps]
        y1 = df['b'].values[i: i + time_steps]
        z1 = df['c'].values[i: i + time_steps]
        x2 = df['d'].values[i: i + time_steps]
        y2 = df['e'].values[i: i + time_steps]
        z2 = df['f'].values[i: i + time_steps]
        x3 = df['g'].values[i: i + time_steps]
        y3 = df['h'].values[i: i + time_steps]
        z3 = df['i'].values[i: i + time_steps]
        x4 = df['j'].values[i: i + time_steps]
        y4 = df['k'].values[i: i + time_steps]
        z4 = df['l'].values[i: i + time_steps]
        x5 = df['m'].values[i: i + time_steps]
        y5 = df['n'].values[i: i + time_steps]
        z5 = df['o'].values[i: i + time_steps]
        x6 = df['p'].values[i: i + time_steps]
        y6 = df['q'].values[i: i + time_steps]
        z6 = df['r'].values[i: i + time_steps]
        x7 = df['s'].values[i: i + time_steps]
        y7 = df['t'].values[i: i + time_steps]
        z7 = df['u'].values[i: i + time_steps]
        x8 = df['v'].values[i: i + time_steps]
        y8 = df['w'].values[i: i + time_steps]
        z8 = df['x'].values[i: i + time_steps]
        x9 = df['y'].values[i: i + time_steps]
        y9 = df['z'].values[i: i + time_steps]
        z9 = df['aa'].values[i: i + time_steps]
        x10 = df['bb'].values[i: i + time_steps]
        y10= df['cc'].values[i: i + time_steps]
        z10 = df['dd'].values[i: i + time_steps]
        x11 = df['ee'].values[i: i + time_steps]
        y11 = df['ff'].values[i: i + time_steps]
        z11 = df['gg'].values[i: i + time_steps]
        x12 = df['hh'].values[i: i + time_steps]
        y12 = df['ii'].values[i: i + time_steps]
        z12 = df['jj'].values[i: i + time_steps]
        x13 = df['kk'].values[i: i + time_steps]
        y13 = df['ll'].values[i: i + time_steps]
        z13 = df['mm'].values[i: i + time_steps]
        x14 = df['nn'].values[i: i + time_steps]
        y14 = df['oo'].values[i: i + time_steps]
        z14 = df['pp'].values[i: i + time_steps]
        x15 = df['qq'].values[i: i + time_steps]
        y15 = df['rr'].values[i: i + time_steps]
        z15 = df['ss'].values[i: i + time_steps]
        x16 = df['tt'].values[i: i + time_steps]
        y16 = df['uu'].values[i: i + time_steps]
        z16 = df['vv'].values[i: i + time_steps]
        x17 = df['ww'].values[i: i + time_steps]
        y17 = df['xx'].values[i: i + time_steps]
        z17 = df['yy'].values[i: i + time_steps]
        x18 = df['zz'].values[i: i + time_steps]
        y18 = df['aaa'].values[i: i + time_steps]
        z18 = df['bbb'].values[i: i + time_steps]
        
        # Retrieve the most often used label in this segment
       
        segments.append([ x1 ,y1 ,z1,x2 ,y2 ,z2 ,x3,y3,z3 ,x4 , y4 , z4 ,x5 ,y5 ,z5 ,x6, 
        y6 ,z6 ,x7 ,y7 ,z7 ,x8 ,y8 , z8 , x9 , y9 , z9 , x10 , y10, z10 , x11 , y11 , z11 , x12, y12 , z12 , x13 , y13 , z13 , x14, 
        y14 ,z14 ,x15 , y15,z15 , x16 ,y16 ,z16,x17,y17 ,z17 ,x18 ,y18 ,z18 ])
      

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1,  N_FEATURES)
    

    return reshaped_segments, labels

x_set_train, y_set_train = create_segments_and_labels(trainAttrX,1,1)

In [0]:
def create_segments_and_labels1(df, time_steps, step):

    # x, y, z acceleration as features
    N_FEATURES = 54
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df), step):
        x1 = df['a'].values[i: i + time_steps]
        y1 = df['b'].values[i: i + time_steps]
        z1 = df['c'].values[i: i + time_steps]
        x2 = df['d'].values[i: i + time_steps]
        y2 = df['e'].values[i: i + time_steps]
        z2 = df['f'].values[i: i + time_steps]
        x3 = df['g'].values[i: i + time_steps]
        y3 = df['h'].values[i: i + time_steps]
        z3 = df['i'].values[i: i + time_steps]
        x4 = df['j'].values[i: i + time_steps]
        y4 = df['k'].values[i: i + time_steps]
        z4 = df['l'].values[i: i + time_steps]
        x5 = df['m'].values[i: i + time_steps]
        y5 = df['n'].values[i: i + time_steps]
        z5 = df['o'].values[i: i + time_steps]
        x6 = df['p'].values[i: i + time_steps]
        y6 = df['q'].values[i: i + time_steps]
        z6 = df['r'].values[i: i + time_steps]
        x7 = df['s'].values[i: i + time_steps]
        y7 = df['t'].values[i: i + time_steps]
        z7 = df['u'].values[i: i + time_steps]
        x8 = df['v'].values[i: i + time_steps]
        y8 = df['w'].values[i: i + time_steps]
        z8 = df['x'].values[i: i + time_steps]
        x9 = df['y'].values[i: i + time_steps]
        y9 = df['z'].values[i: i + time_steps]
        z9 = df['aa'].values[i: i + time_steps]
        x10 = df['bb'].values[i: i + time_steps]
        y10= df['cc'].values[i: i + time_steps]
        z10 = df['dd'].values[i: i + time_steps]
        x11 = df['ee'].values[i: i + time_steps]
        y11 = df['ff'].values[i: i + time_steps]
        z11 = df['gg'].values[i: i + time_steps]
        x12 = df['hh'].values[i: i + time_steps]
        y12 = df['ii'].values[i: i + time_steps]
        z12 = df['jj'].values[i: i + time_steps]
        x13 = df['kk'].values[i: i + time_steps]
        y13 = df['ll'].values[i: i + time_steps]
        z13 = df['mm'].values[i: i + time_steps]
        x14 = df['nn'].values[i: i + time_steps]
        y14 = df['oo'].values[i: i + time_steps]
        z14 = df['pp'].values[i: i + time_steps]
        x15 = df['qq'].values[i: i + time_steps]
        y15 = df['rr'].values[i: i + time_steps]
        z15 = df['ss'].values[i: i + time_steps]
        x16 = df['tt'].values[i: i + time_steps]
        y16 = df['uu'].values[i: i + time_steps]
        z16 = df['vv'].values[i: i + time_steps]
        x17 = df['ww'].values[i: i + time_steps]
        y17 = df['xx'].values[i: i + time_steps]
        z17 = df['yy'].values[i: i + time_steps]
        x18 = df['zz'].values[i: i + time_steps]
        y18 = df['aaa'].values[i: i + time_steps]
        z18 = df['bbb'].values[i: i + time_steps]
        
        # Retrieve the most often used label in this segment
       
        segments.append([ x1 ,y1 ,z1,x2 ,y2 ,z2 ,x3,y3,z3 ,x4 , y4 , z4 ,x5 ,y5 ,z5 ,x6, 
        y6 ,z6 ,x7 ,y7 ,z7 ,x8 ,y8 , z8 , x9 , y9 , z9 , x10 , y10, z10 , x11 , y11 , z11 , x12, y12 , z12 , x13 , y13 , z13 , x14, 
        y14 ,z14 ,x15 , y15,z15 , x16 ,y16 ,z16,x17,y17 ,z17 ,x18 ,y18 ,z18 ])
      

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_FEATURES)
    

    return reshaped_segments, labels

x_set_test, y_set_test = create_segments_and_labels1(testAttrX,1,1)

In [0]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D

In [0]:
visible = Input(shape=(224,224,3))
visible2 = Input(shape=(54,))

In [0]:
#First neural network 

conv11 = Conv2D(96, kernel_size=4, activation='relu')(visible)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
batch11 = BatchNormalization()(pool11)

conv12 = Conv2D(256, kernel_size=4, activation='relu')(batch11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
batch12 = BatchNormalization()(pool12)

conv13 = Conv2D(256, kernel_size=4, activation='relu')(batch12)
pool13 = MaxPooling2D(pool_size=(2, 2))(conv13)
batch13 = BatchNormalization()(pool13)



flat1 = Flatten()(batch13)
hidden1 = Dense(64, activation='relu')(flat1)
hidden2 = Dense(128, activation='relu')(hidden1)



hidden21 = Dense(64, activation='relu')(visible2)
hidden22 = Dense(128, activation='relu')(hidden21)


In [0]:
merge = concatenate([hidden2, hidden22], axis=-1)

In [0]:
merge.shape

TensorShape([Dimension(None), Dimension(256)])

In [0]:
hidden123 = Dense(128, activation='relu')(merge)
hidden223 = Dense(256, activation='relu')(hidden123)
output = Dense(4, activation='softmax')(hidden223)
model = Model(inputs=[visible, visible2], outputs=output)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 221, 221, 96) 4704        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 110, 110, 96) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 110, 110, 96) 384         max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
history = model.fit([trainImagesX, x_set_train],labels_train, batch_size=8,validation_split=0.1,   epochs=25, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 226 samples, validate on 26 samples
Epoch 1/25
226/226 [==============================] - 213s 944ms/step - loss: 6.8892 - acc: 0.2478 - val_loss: 11.2692 - val_acc: 0.2692
Epoch 2/25
226/226 [==============================] - 212s 938ms/step - loss: 12.2297 - acc: 0.2389 - val_loss: 11.7786 - val_acc: 0.2692
Epoch 3/25
226/226 [==============================] - 212s 939ms/step - loss: 12.0529 - acc: 0.2522 - val_loss: 11.7786 - val_acc: 0.2692
Epoch 4/25
226/226 [==============================] - 211s 935ms/step - loss: 12.0529 - acc: 0.2522 - val_loss: 11.7786 - val_acc: 0.2692
Epoch 5/25
226/226 [==============================] - 211s 935ms/step - loss: 12.0529 - acc: 0.2522 - val_loss: 11.7786 - val_acc: 0.2692
Epoch 6/25
226/226 [==============================] - 211s 934ms/step - loss: 12.0529 - acc: 0.2522 - val_loss: 11.7786 - val_acc: 0.2692
Epoch 7/25
226/226 [============